### Loading libraries

In [ ]:
library(tidyverse)
library(tidymodels)
library(gridExtra)

### Utility functions

In [ ]:
fig <- function(width, heigth){
     options(repr.plot.width = width, repr.plot.height = heigth)
}

### Loading data

In [ ]:
path <- ""

names <- read.table(paste(path, "spotify-names.txt", sep = ""),header = TRUE)
songs <- read.table(paste(path, "spotify-extr.txt", sep = ""),
                    sep = " ", header = TRUE) %>%
    as_tibble() %>%
    mutate(name = names$x,
           key = factor(key),
           mode = factor(mode),
           pop.class = factor(pop.class)) %>%
    relocate(c(pop.class, popularity)) %>%
    relocate(c(key, mode, name), .after=last_col())

songs.quant <- songs %>% select(popularity:tempo)

head(songs)

## Exploratory statistics

In [ ]:
fig(12,8)
songs.quant %>%
    #filter(speechiness < .75) %>%
    mutate(duration=log(duration)) %>%
    pivot_longer(cols=everything(), names_to='variable', values_to='value') %>%
ggplot() +
    geom_histogram(aes(value), fill='#2FD565', color='#000000', bins=30) +
    facet_wrap(~variable, scales='free')

In [ ]:
songs %>%
    filter(instrumentalness > .75) #%>% ggplot() + geom_bar(stat='count', aes(pop.class))

In [ ]:
fig(15,10)
songs.quant %>%
    mutate(duration=log(duration)) %>%
    pivot_longer(cols=!popularity, names_to='variable', values_to='value') %>%
ggplot() +
    geom_point(aes(value, popularity), size=.7, alpha=.3) + 
    facet_wrap(~variable, scales='free')

In [ ]:
fig(15,10)
songs.quant %>%
    mutate(duration=log(duration)) %>%
    pivot_longer(cols=!year, names_to='variable', values_to='value') %>%
ggplot() +
    geom_point(aes(value, year), size=.7, alpha=.3) + 
    facet_wrap(~variable, scales='free')

In [ ]:
fig(15,10)
songs %>%
    mutate(duration=log(duration)) %>%
    select(c(pop.class, valence:tempo)) %>%
    pivot_longer(cols=!pop.class, names_to='variable', values_to='value') %>%
ggplot() +
    #geom_jitter(aes(value, pop.class), size=.7, alpha=.3) + 
    geom_boxplot(aes(value, pop.class)) +
    facet_wrap(~variable, scales='free') +
    scale_y_discrete(limits=rev)

In [ ]:
library(corrplot)
cormat <- cor(songs.quant)
corrplot(cormat, method="ellipse")

In [ ]:
fig(8,6)
songs %>%
ggplot() +
    geom_boxplot(aes(x=reorder(key, -popularity, FUN=median), y=popularity), fill='#2FD565', color='#000000') +
    labs(x='Key', y='Popularity')


songs %>%
ggplot() +
    geom_bar(stat='count', aes(reorder(key, -popularity, FUN=median)), fill='#2FD565', color='#000000') +
    labs(x='Key', y='Count')

In [ ]:
fig(8,6)
songs %>%
ggplot() +
    geom_boxplot(aes(mode, popularity), fill='#2FD565', color='#000000') +
    labs(x='Mode', y='Popularity')

In [ ]:
library(ggmosaic)
songs %>%
ggplot() +
    geom_mosaic(aes(product(mode, key)), fill='#2FD565')

## Principal component analysis

In [ ]:
library(FactoMineR)
library(factoextra)
res.pca <- songs.quant %>% 
    select(!popularity) %>% 
    PCA(ncp=11)

In [ ]:
fig(12,5)


pc_eig <- tibble(pc=c(1:11), eig=res.pca$eig[1:11,2], cumeig=res.pca$eig[1:11,3])
g1 <- pc_eig %>%
ggplot(aes(pc, eig)) + 
    geom_bar(stat="identity", fill='#2FD565') +
    geom_line() + 
    geom_point() +
    scale_x_continuous(breaks=c(1:11), labels=paste('PC', c(1:11), sep=''), minor_breaks=NULL) +
    labs(x='', y='Percentage of variance', title='Percentage of variance by principal component') +
    annotate('text', x=c(1:11)+.4, y=pc_eig$eig+1.1, label=paste(round(pc_eig$eig,1), '%', sep='')) +
    theme(text=element_text(size=14))

g2 <- pc_eig %>%
ggplot(aes(pc, cumeig)) + 
    geom_bar(stat="identity", fill='#2FD565') +
    geom_line() + 
    geom_point() +
    scale_x_continuous(breaks=c(1:11), labels=paste('PC', c(1:11), sep=''), minor_breaks=NULL) +
    labs(x='', y='Percentage of variance', title='Cumulative percentage of variance') +
    #annotate('text', x=c(1:11)-.1, y=pc_eig$cumeig+6, label=paste(round(pc_eig$cumeig,1), '%', sep='')) +
    theme(text=element_text(size=14))

grid.arrange(g1, g2, nrow=1)

In [ ]:
pc_eig

In [ ]:
fig(8,6)
fviz_pca_var(res.pca, col.var="contrib") +
    scale_color_gradient(low="black", high="green") +
    labs(x='PC1 (31.3%)', y='PC2 (14.9%)', color='Contribution') +
    guides(color='none') +
    theme_gray() + theme(text=element_text(size=14))

In [ ]:
name_func <- function(name) {
    paste('PC', substring(name, first=5), sep='')
}

pca.ind <- as_tibble(res.pca$ind$coord) %>%
    rename_with(name_func, everything()) %>%
    mutate(pop.class=songs$pop.class)

gg_color_hue <- function(n) {
  hues = seq(15, 375, length = n + 1)
  hcl(h = hues, l = 65, c = 100)[1:n]
}
colors <- gg_color_hue(4)

fig(10,8)
ggplot(pca.ind) + 
    geom_point(aes(x=PC1, y=PC2, color=pop.class), alpha=.5, shape=19) +
    scale_x_continuous(limits=c(-6,4)) +
    scale_y_continuous(limits=c(-5,5)) +
    guides(color = guide_legend(override.aes = list(size = 10))) +
    labs(x='PC1 (31.3%)', y='PC2 (14.9%)', color="Popularity class") +
    theme(text=element_text(size=14))

# Classification models

In [ ]:
train_ratio <- .75 
data_split <- songs %>%
    select(!c(popularity, name)) %>%
    initial_split(strata = pop.class, prop = train_ratio)

songs_train <- training(data_split)
songs_test <- testing(data_split)

## Logistic regression

In [ ]:
library(glmnet)

### Without regularization

In [ ]:
log_reg <- multinom_reg(penalty = 0) %>%
    set_engine('glmnet')

In [ ]:
log_reg_fit <-
    log_reg %>%
    set_engine('glmnet') %>%
    fit(pop.class ~ ., 
        data=songs_train)

#tidy(log_reg_fit) %>% filter(estimate > 0)

In [ ]:
log_reg_results <- 
  songs_test %>%
  select(pop.class) %>%
  bind_cols(
    predict(log_reg_fit, new_data = songs_test %>% select(-pop.class))
  )

In [ ]:
log_reg_results %>%
    conf_mat(truth=pop.class, estimate=.pred_class)

log_reg_results %>%
    precision(pop.class, .pred_class)

### With regularization

In [ ]:
log_reg <- multinom_reg(penalty = .005, mixture=1) %>%
    set_engine('glmnet')

In [ ]:
log_reg_fit <-
    log_reg %>%
    set_engine('glmnet') %>%
    fit(pop.class ~ (.)^2, 
        data=songs_train)

tidy(log_reg_fit) %>% filter(estimate > 0)

In [ ]:
log_reg_results <- 
  songs_test %>%
  select(pop.class) %>%
  bind_cols(
    predict(log_reg_fit, new_data = songs_test %>% select(-pop.class))
  )

In [ ]:
log_reg_results %>%
    conf_mat(truth=pop.class, estimate=.pred_class)

log_reg_results %>%
    precision(pop.class, .pred_class)

In [ ]:
log_reg <- multinom_reg(penalty = tune()) %>%
    set_engine('glmnet')

In [ ]:
songs_rec <- recipe(pop.class ~ ., data = songs_train)# %>%
  #step_normalize(all_numeric(), -all_outcomes())

songs_prep <- songs_rec %>%
  prep()

In [ ]:
lasso_spec <- multinom_reg(penalty = 0.1, mixture = 1) %>%
  set_engine("glmnet")

wf <- workflow() %>%
  add_recipe(songs_rec)

#lasso_fit <- wf %>%
#  add_model(lasso_spec) %>%
#  fit(data = songs_train)

#lasso_fit %>%
#  pull_workflow_fit() %>%
#  tidy()

## SMV

In [ ]:
svm_mod <-
  svm_rbf(cost = tune(), rbf_sigma = tune()) %>%
  set_mode("classification") %>%
  set_engine("kernlab")

In [ ]:
songs_rec <-
  recipe(pop.class ~ ., data = songs_train)  %>%
  # remove any zero variance predictors
  step_zv(all_predictors()) %>% 
  # remove any linear combinations
  step_lincomb(all_numeric())

In [ ]:
songs_rs <- rsample::bootstraps(songs_train, times = 2)

In [ ]:
roc_vals <- metric_set(roc_auc)
ctrl <- control_grid(verbose = FALSE, save_pred = TRUE)

In [ ]:
formula_res1 <-
  svm_mod %>% 
  tune_grid(
    pop.class ~ .,
    resamples = songs_rs,
#    metrics = accuracy,
    control = ctrl
  )
#formula_res

In [ ]:
formula_res %>% 
  select(.metrics) %>% 
  slice(1) %>% 
  pull(1)

In [ ]:
estimates <- collect_metrics(formula_res)
estimates

In [ ]:
svm_results <- 
  songs_test %>%
  select(pop.class) %>%
  bind_cols(
    predict(s_fit, new_data = songs_test %>% select(-pop.class))
  )

## Random forest

In [ ]:
library(ranger)

In [ ]:
rf_mod <- ranger(pop.class ~ ., data=songs_train, num.trees=500, verbose=TRUE)

In [ ]:
attributes(rf_mod)

In [ ]:
rf_mod$predictions

In [ ]:
rf_results <- 
  songs_test %>%
  select(pop.class) %>%
  bind_cols(
    predict(rf_fit, new_data = songs_test %>% select(-pop.class))
  )

rf_results %>%
    conf_mat(truth=pop.class, estimate=.pred_class)

rf_results %>%
    precision(pop.class, .pred_class)